In [1]:
from huggingface_hub import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# Choose your model (e.g., TinyLlama)
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
local_dir = "./model"  # Where to save

# Download and save
snapshot_download(repo_id=model_name, local_dir=local_dir)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

In [2]:
local_model_path = "./model"  # Path to your saved model

tokenizer = AutoTokenizer.from_pretrained(local_model_path)
model = AutoModelForCausalLM.from_pretrained(
    local_model_path,
    device_map="cpu",
    torch_dtype=torch.float32
)

In [3]:
# Create optimized pipeline
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    # device="cpu",  # Explicit CPU (ValueError: The model has been loaded with `accelerate` and therefore cannot be moved to a specific device. Please discard the `device` argument when creating your pipeline object)
    torch_dtype=torch.float32,
)

In [4]:
def generate_response(prompt):
    formatted_prompt = f"""<|system|>
    You are a coding assistant for Python, Linux, and ML. Give short answers.</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    """
    output = pipe(
        formatted_prompt,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_k=50,
        pad_token_id=pipe.tokenizer.eos_token_id
    )
    return output[0]["generated_text"].split("<|assistant|>")[-1].strip()


In [5]:
print("Ask about coding (type 'exit'):")
while True:
    user_input = input("> ")
    if user_input.lower() == "exit":
        break
    print(f"Question: {user_input}")
    print(f"Answer:\n{generate_response(user_input)}")
    print(10*"-")
    

Ask about coding (type 'exit'):
Question: What is a linear regression?
Answer:
Linear regression is a statistical model that estimates the relationship between a dependent variable (e.g. Y) and a set of independent variables (x). The dependent variable represents the outcome of an event or the outcome of a business or product. The independent variables are the factors that affect the outcome and include things like age, gender, income, education, or other factors. The model is represented by a line that best predicts the y value(s) given the x values. Linear regression can be used to estimate the relationship between a set of variables and to find the values of the independent variables that willющеve the best fit.
----------
